In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence

# def unet_model(input_size=(IMG_HEIGHT, IMG_WIDTH, 3)):
def build_unet_model(hp):

  # Hyperparameters to tune
  dropout_rate = hp.Float('dropout_rate', 0.05, 0.3, step=0.05)
  learning_rate = hp.Float('learning_rate', 1e-5, 1e-2, sampling='log')
  base_filters = hp.Choice('base_filters', [8, 16, 32])
  use_batch_norm = hp.Boolean('use_batch_norm')

  input_size = (IMG_HEIGHT, IMG_WIDTH, 3)

  # Build model
  inputs = Input(input_size)

  # Encoder
  c1 = Conv2D(base_filters, (3, 3), activation='relu', padding='same')(inputs)
  if use_batch_norm:
    c1 = BatchNormalization()(c1)
  c1 = Conv2D(base_filters, (3, 3), activation='relu', padding='same')(c1)
  if use_batch_norm:
    c1 = BatchNormalization()(c1)
  p1 = MaxPooling2D((2, 2))(c1)
  p1 = Dropout(dropout_rate)(p1)

  c2 = Conv2D(base_filters * 2, (3, 3), activation='relu', padding='same')(p1)
  if use_batch_norm:
    c2 = BatchNormalization()(c2)
  c2 = Conv2D(base_filters * 2, (3, 3), activation='relu', padding='same')(c2)
  if use_batch_norm:
    c2 = BatchNormalization()(c2)
  p2 = MaxPooling2D((2, 2))(c2)
  p2 = Dropout(dropout_rate)(p2)

  c3 = Conv2D(base_filters * 4, (3, 3), activation='relu', padding='same')(p2)
  if use_batch_norm:
    c3 = BatchNormalization()(c3)
  c3 = Conv2D(base_filters * 4, (3, 3), activation='relu', padding='same')(c3)
  if use_batch_norm:
    c3 = BatchNormalization()(c3)
  p3 = MaxPooling2D((2, 2))(c3)
  p3 = Dropout(dropout_rate)(p3)

  # Bottleneck
  c4 = Conv2D(base_filters * 8, (3, 3), activation='relu', padding='same')(p3)
  if use_batch_norm:
    c4 = BatchNormalization()(c4)
  c4 = Conv2D(base_filters * 8, (3, 3), activation='relu', padding='same')(c4)
  if use_batch_norm:
    c4 = BatchNormalization()(c4)
  c4 = Dropout(dropout_rate * 1.5)(c4)  # Higher dropout in bottleneck

  # Decoder
  u1 = UpSampling2D((2, 2))(c4)
  u1 = Concatenate()([u1, c3])
  c5 = Conv2D(base_filters * 4, (3, 3), activation='relu', padding='same')(u1)
  if use_batch_norm:
    c5 = BatchNormalization()(c5)
  c5 = Conv2D(base_filters * 4, (3, 3), activation='relu', padding='same')(c5)
  if use_batch_norm:
    c5 = BatchNormalization()(c5)
  c5 = Dropout(dropout_rate)(c5)

  u2 = UpSampling2D((2, 2))(c5)
  u2 = Concatenate()([u2, c2])
  c6 = Conv2D(base_filters * 2, (3, 3), activation='relu', padding='same')(u2)
  if use_batch_norm:
    c6 = BatchNormalization()(c6)
  c6 = Conv2D(base_filters * 2, (3, 3), activation='relu', padding='same')(c6)
  if use_batch_norm:
    c6 = BatchNormalization()(c6)
  c6 = Dropout(dropout_rate)(c6)

  u3 = UpSampling2D((2, 2))(c6)
  u3 = Concatenate()([u3, c1])
  c7 = Conv2D(base_filters, (3, 3), activation='relu', padding='same')(u3)
  if use_batch_norm:
    c7 = BatchNormalization()(c7)
  c7 = Conv2D(base_filters, (3, 3), activation='relu', padding='same')(c7)
  if use_batch_norm:
    c7 = BatchNormalization()(c7)
  c7 = Dropout(dropout_rate)(c7)
  
  outputs = Conv2D(1, (1, 1), activation='sigmoid')(c7)

  model = Model(inputs, outputs)
  
  model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy', iou_metric, precision_metric, recall_metric, f1_metric]
  )
  
  return model

2025-06-20 10:06:47.352073: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750414007.371398 4190628 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750414007.377350 4190628 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750414007.391807 4190628 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750414007.391826 4190628 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750414007.391828 4190628 computation_placer.cc:177] computation placer alr

In [2]:
import keras_tuner as kt

def run_hyperparameter_search(model, x_train, y_train, x_val, y_val):
  """Run hyperparameter search"""
  
  # Initialize tuner
  tuner = kt.RandomSearch(
      model,
      objective=kt.Objective("val_iou_metric", direction="max"),  # Or 'val_loss', 'val_precision', etc.
      max_trials=20,  # Number of different hyperparameter combinations to try
      directory='unet_tuning',
      project_name='water_detection'
  )
  
  # Search for best hyperparameters
  tuner.search(
      x_train, y_train,
      epochs=20,  # Fewer epochs per trial to speed up search
      validation_data=(x_val, y_val),
      batch_size=32,
      verbose=1
  )
  
  # Get best hyperparameters
  best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
  
  print(f"Best dropout rate: {best_hps.get('dropout_rate')}")
  print(f"Best learning rate: {best_hps.get('learning_rate')}")
  print(f"Best base filters: {best_hps.get('base_filters')}")
  print(f"Use batch norm: {best_hps.get('use_batch_norm')}")
  
  # Build and train final model with best hyperparameters
  final_model = tuner.hypermodel.build(best_hps)
  
  return final_model, best_hps

Model after applying the tuned hyperparameters

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence

print(tf.config.list_physical_devices('GPU'))

# def unet_model(input_size=(IMG_HEIGHT, IMG_WIDTH, 3)):
def unet_model(input_size, dropout_rate=0.05):
  inputs = Input(input_size)

  # Encoder
  c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
  c1 = BatchNormalization()(c1)
  c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
  c1 = BatchNormalization()(c1)
  p1 = MaxPooling2D((2, 2))(c1)
  p1 = Dropout(dropout_rate)(p1)

  c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
  c2 = BatchNormalization()(c2)
  c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
  c2 = BatchNormalization()(c2)
  p2 = MaxPooling2D((2, 2))(c2)
  p2 = Dropout(dropout_rate)(p2)

  c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
  c3 = BatchNormalization()(c3)
  c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
  c3 = BatchNormalization()(c3)
  p3 = MaxPooling2D((2, 2))(c3)
  p3 = Dropout(dropout_rate)(p3)

  # Bottleneck
  c4 = Conv2D(256, (3, 3), activation='relu', padding='same')(p3)
  c4 = BatchNormalization()(c4)
  c4 = Conv2D(256, (3, 3), activation='relu', padding='same')(c4)
  c4 = BatchNormalization()(c4)
  c4 = Dropout(dropout_rate * 1.5)(c4)  # Higher dropout in bottleneck

  # Decoder
  u1 = UpSampling2D((2, 2))(c4)
  u1 = Concatenate()([u1, c3])
  c5 = Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
  c5 = BatchNormalization()(c5)
  c5 = Conv2D(128, (3, 3), activation='relu', padding='same')(c5)
  c5 = BatchNormalization()(c5)
  c5 = Dropout(dropout_rate)(c5)

  u2 = UpSampling2D((2, 2))(c5)
  u2 = Concatenate()([u2, c2])
  c6 = Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
  c6 = BatchNormalization()(c6)
  c6 = Conv2D(64, (3, 3), activation='relu', padding='same')(c6)
  c6 = BatchNormalization()(c6)
  c6 = Dropout(dropout_rate)(c6)

  u3 = UpSampling2D((2, 2))(c6)
  u3 = Concatenate()([u3, c1])
  c7 = Conv2D(32, (3, 3), activation='relu', padding='same')(u3)
  c7 = BatchNormalization()(c7)
  c7 = Conv2D(32, (3, 3), activation='relu', padding='same')(c7)
  c7 = BatchNormalization()(c7)
  c7 = Dropout(dropout_rate)(c7)
  
  outputs = Conv2D(1, (1, 1), activation='sigmoid')(c7)

  model = Model(inputs, outputs)
  return model

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
